# CAD PROJECT- Classical Approach
*By Jaime Simarro & Ahmed Gouda*

In [0]:
from google.colab import drive

drive.mount("/content/drive/", force_remount=True)

Mounted at /content/drive/


In [0]:
import numpy as np
import cv2
import os
import glob
import matplotlib.pyplot as plt
import math
import numpy.ma as ma
!pip install ciecam02
from ciecam02 import rgb2jch, jch2rgb
from skimage.feature import greycomatrix, greycoprops


# SETUP PARAMETERS

In [0]:
#### JUST CHANGE THIS #####

dermo_challenge=True
number_Normalization_type=1
number_Feat_type=[1]
number_Windowing_type = 2


number_Dataset_type = 1 #1: train, 2:val 3:test
###########################
switcher_window = {
    1: "One",
    2: "Two",
}

switcher_normalization = {
    1: "Original",
    2: "Reinhard",
}
switcher_feat = {
    1: "GLCM",
    2: "Histogram",
    3: "Fintesity",
    4: "Fstat",
    5: "Gabour",
}
normalization_type=switcher_normalization.get(number_Normalization_type, "Invalid normalization type")
windowing_type=switcher_window.get(number_Windowing_type, "Invalid windowing type")

In [0]:
#Load database directory
if (dermo_challenge):
  if(number_Dataset_type == 1):
    path_data_pos='/content/drive/My Drive/Colab Notebooks/CAD Project/Dermo Challenge/train/nv/*.jpg'  # Nevus
    path_data_neg='/content/drive/My Drive/Colab Notebooks/CAD Project/Dermo Challenge/train/les/*.jpg' # Others
  if(number_Dataset_type == 2):
    path_data_pos='/content/drive/My Drive/Colab Notebooks/CAD Project/Dermo Challenge/val/nv/*.jpg'  # Nevus
    path_data_neg='/content/drive/My Drive/Colab Notebooks/CAD Project/Dermo Challenge/val/les/*.jpg' # Others
  if(number_Dataset_type == 3):
    path_data= '/content/drive/My Drive/Colab Notebooks/CAD Project/Dermo Challenge/test/*.jpg' # Others
    path_general_feat='/content/drive/My Drive/Colab Notebooks/CAD Project/Dermo Features/'
else:
  if number_Normalization_type==1: #Original
    if(number_Dataset_type == 1):
      path_data_pos='/content/drive/My Drive/Colab Notebooks/CAD Project/Histopathology Challenge/train/m0/*.png' # Malignan
      path_data_neg='/content/drive/My Drive/Colab Notebooks/CAD Project/Histopathology Challenge/train/b0/*.png' # Belignan
    if(number_Dataset_type == 2):
      path_data_pos='/content/drive/My Drive/Colab Notebooks/CAD Project/Histopathology Challenge/val/m0/*.png' # Malignan
      path_data_neg='/content/drive/My Drive/Colab Notebooks/CAD Project/Histopathology Challenge/val/b0/*.png' # Belignan
    if(number_Dataset_type == 3):
      path_data='/content/drive/My Drive/Colab Notebooks/CAD Project/Histopathology Challenge/test/*.png' # Belignan
  if number_Normalization_type==2: #Reinhard
    if(number_Dataset_type == 1):
      path_data_pos='/content/drive/My Drive/Colab Notebooks/CAD Project/Histopathology Challenge Reinhard/train/m0/*.png' # Malignan
      path_data_neg='/content/drive/My Drive/Colab Notebooks/CAD Project/Histopathology Challenge Reinhard/train/b0/*.png' # Belignan
    if(number_Dataset_type == 2):
      path_data_pos='/content/drive/My Drive/Colab Notebooks/CAD Project/Histopathology Challenge Reinhard/val/m0/*.png' # Malignan
      path_data_neg='/content/drive/My Drive/Colab Notebooks/CAD Project/Histopathology Challenge Reinhard/val/b0/*.png' # Belignan
    if(number_Dataset_type == 3):
      path_data='/content/drive/My Drive/Colab Notebooks/CAD Project/Histopathology Challenge Reinhard/test/*.png' # Belignan

  path_general_feat='/content/drive/My Drive/Colab Notebooks/CAD Project/Histopathology Features/'

# Preprocessing


In [0]:
#Function to remove the cornner artifact
def remove_black_focus(img):
    threshold=30
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    gray = cv2.GaussianBlur(gray, (17, 17), 32)
    new_img=img.copy()   
    try:
        y1=np.max(np.where(gray[:150,:200]<threshold)[0])
        x1=np.max(np.where(gray[:150,:200]<threshold)[1])
        new_img=new_img[y1:,x1:, :]
    except:
        pass
    try:
        x2=np.min(np.where(gray[300:,400:]<threshold)[1])
        y2=np.min(np.where(gray[300:,400:]<threshold)[0])
        new_img=new_img[:300+y2,:400+x2, :]
    except:
        pass    
    return cv2.resize(new_img,(img.shape[1],img.shape[0]))

def histogram_stretching(img):
    if(img.ndim==2):
        img=(img-img.min())/(img.max()-img.min())*255
    else:
        for c in range(img.shape[2]):
            chanel=img[:,:,c]
            img[:,:,c]=(chanel-chanel.min())/(chanel.max()-chanel.min())*255
    return img.astype(np.uint8)

# FEATURE EXTRACTION

## Color spaces


1.   Dermo
  *   Six melanoma colors [1]
  *   Hybrid color space [2]
2.   Histopathology 
  *  Segmentaion in 4 broad tissue classes corresponding to nuclei, epithelium, stroma and backgroundtion [3] We have tried calculate the distance in Lab and Luv because they are Linear color spaces. For Luv epithelium is not well calculated (color white).

[1] KASMI, Reda; MOKRANI, Karim. Classification of malignant melanoma and benign skin lesions: implementation of automatic ABCD rule. IET Image Processing, 2016, vol. 10, no 6, p. 448-455.

[2] OGHAZ, Mahdi Maktabdar, et al. A hybrid color space for skin detection using genetic algorithm heuristic search and principal component analysis technique. PloS one, 2015, vol. 10, no 8, p. e0134828.

[3] BASAVANHALLY, Ajay; MADABHUSHI, Anant. EM-based segmentation-driven color standardization of digitized histopathology. En Medical Imaging 2013: Digital Pathology. International Society for Optics and Photonics, 2013. p. 86760G.



In [0]:
import sklearn.mixture as mix
from sklearn.cluster import KMeans

def six_melanoma_colors(img):
  # BGR
  white=(217,188,197)
  black=(30,31,41)
  red=(17,21,118)
  l_brown=(16,82,163)
  d_brown=(5,44,135)
  blue_gray=(139,108,113)
  w=np.linalg.norm(np.subtract(img,white), axis=2)
  b=np.linalg.norm(np.subtract(img,black), axis=2)
  r=np.linalg.norm(np.subtract(img,red), axis=2)
  lb=np.linalg.norm(np.subtract(img,l_brown), axis=2)
  db=np.linalg.norm(np.subtract(img,d_brown), axis=2)
  bg=np.linalg.norm(np.subtract(img,blue_gray), axis=2)
  I=np.stack((w,b,r,lb,db,bg),axis=2)
  return histogram_stretching(I)


def hybrid_color_space(img):
  #https://www.ncbi.nlm.nih.gov/pmc/articles/PMC4534136/
  hybrid=np.empty(img.shape)
  g=img[:,:,0]
  b=img[:,:,1]
  r=img[:,:,2]
  div_g=np.divide(g,np.sum(g+b+r+1))
  hybrid[:,:,0]=0.088*div_g-58.89*g-30.014*r-11.95*b-7.24
  hybrid[:,:,1]=2.122*b+14.859*g+6.921*r+0.62*div_g+1.744
  hybrid[:,:,2]=0.342*div_g-3.698*g-2.25*r-0.103*b-0.464
  return histogram_stretching(hybrid)


def tissue_segmentation(imgLab):
    # Template image: m0/m_trainImage23498.png
    template_centersLab=[[ 45.89194057, 155.47816299, 116.56506078],
                      [215.95958188, 145.9554007 , 124.14634146],
                      [103.41602634, 160.0640322 , 120.28320527], 
                      [155.97240891, 157.4849664 , 122.12239123]] 
    img_flat = imgLab.reshape((imgLab.shape[0] *imgLab.shape[1], 3))
    if True:
        gmm=mix.GaussianMixture(n_components=4,random_state=0)
        gmm.fit_predict(img_flat)
        tissues_centers=gmm.means_
    else:
        clt = KMeans(n_clusters = 4,random_state=0)
        labels = clt.fit_predict(img_flat)
        tissues_centers = clt.cluster_centers_
    tissue_class=[]
    for i in range(4): # Select the closest distance
        tissue_class.append(np.argmin(np.linalg.norm(np.subtract(template_centersLab[i],tissues_centers),axis=1)))
    nuclei=tissues_centers[tissue_class[0],:]
    stroma=tissues_centers[tissue_class[1],:]
    epithelium=tissues_centers[tissue_class[2],:]
    background=tissues_centers[tissue_class[3],:]

    n=np.linalg.norm(np.subtract(imgLab,nuclei), axis=2)
    s=np.linalg.norm(np.subtract(imgLab,stroma), axis=2)
    e=np.linalg.norm(np.subtract(imgLab,epithelium), axis=2)
    b=np.linalg.norm(np.subtract(imgLab,background), axis=2)
    segmentanted=np.stack((n,s,e,b),axis=2)
    return histogram_stretching(segmentanted)

## Mask Creation

In [0]:
def create_circular_mask(h, w, center=None, radius=None):
    if center is None: # use the middle of the image
        center = [int(w/2), int(h/2)]
    if radius is None: # use the smallest distance between the center and image walls
        radius = min(center[0], center[1], w-center[0], h-center[1])
    Y, X = np.ogrid[:h, :w]
    dist_from_center = np.sqrt((X - center[0])**2 + (Y-center[1])**2)
    mask = dist_from_center <= radius
    return mask

def create_segmented_ROI(img):
    black=(30,31,41)
    b=np.linalg.norm(np.subtract(img,black), axis=2)
    histogram_stretching(b)   
    blur = cv2.GaussianBlur(b, (17, 17), 32)
    blur = np.uint8(blur)
    _,thresh = cv2.threshold(blur,0,255,cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU)
    _, contours,_ = cv2.findContours(thresh, cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
    distance=[]
    for i in range(len(contours)):
        M = cv2.moments(contours[i])
        if M['m00']!=0 and cv2.contourArea(contours[i])>1000:
            cy = int(M['m10']/M['m00'])
            cx = int(M['m01']/M['m00'])
            distance.append(np.sqrt( (cx-225)**2 + (cy-300)**2 ))
            gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        else:
            distance.append(10000)
            
    mask = np.zeros(b.shape, np.uint8)
    cv2.drawContours(mask, contours, np.argmin(distance), (255),cv2.FILLED)
    return mask

## Features in one channel

In [0]:
import matplotlib.pyplot as plt
import numpy as np
from scipy import ndimage as ndi

from skimage import data
from skimage.util import img_as_float
from skimage.filters import gabor_kernel

# Gabour features - 32
Gabour_kernels = []# prepare filter bank kernels
for theta in range(4):
    theta = theta / 4. * np.pi
    for sigma in (1, 3):
        for frequency in (0.05, 0.25):
            kernel = np.real(gabor_kernel(frequency, theta=theta, sigma_x=sigma, sigma_y=sigma))
            Gabour_kernels.append(kernel)

#Extract Gabor filter in the Normal case
def compute_gabor_feats(image, kernels):
    feats = np.zeros((len(kernels)*2), dtype=np.double)
    for k, kernel in enumerate(kernels):
        filtered = ndi.convolve(image, kernel, mode='wrap')
        feats[k] = filtered.mean();
        feats[k+len(kernels)] = filtered.var();
    return feats

#Extract Gabor filter in the case of Dermo with ROI mask
def compute_gabor_feats_mask(image_ori, kernels, mask_ori):
    scale_percent = 25 # percent of original size
    width = int(image_ori.shape[1] * scale_percent / 100)
    height = int(image_ori.shape[0] * scale_percent / 100)
    dim = (width, height)

    image = cv2.resize(image_ori, dim, interpolation = cv2.INTER_AREA)
    mask_img = cv2.resize(mask_ori, dim, interpolation = cv2.INTER_NEAREST)
    feats = np.zeros((len(kernels)*2), dtype=np.double)
    for k, kernel in enumerate(kernels):
        filtered = ndi.convolve(image, kernel, mode='wrap')
        filtered_compress = ma.array(image,mask=~mask_img).compressed()
        #print(filtered_compress.shape)
        feats[k] = filtered_compress.mean();
        feats[k+len(kernels)] = filtered_compress.var();
    return feats

In [0]:
from scipy.stats import entropy,skew,kurtosis

name_glcm=['dissimilarity','correlation','contrast','homogeneity','ASM','energy']
name_histogram=['h1','h2','h3','h4','h5','h6','h7','h8','h9','h10']
name_intesity=['mean','skew','kurtosis']
name_stat=['smoothness','uniformity','fentropy']
name_feat=name_glcm+name_histogram+name_intesity+name_stat

def one_channel_feat(Channel, feat_type, mask_img = None):
  if (dermo_challenge) and (number_Windowing_type == 2):
    if(feat_type == "Histogram") or (feat_type == "Fintesity")or (feat_type == "Fstat"):
      Channel = ma.array(Channel,mask=~mask_img).compressed()
      #plt.imshow(mask_img,cmap='gray')
      #plt.show()
      #print(Channel.shape)
  if(feat_type == "GLCM"):
    # Glcm features- 6
    glcm = greycomatrix(Channel, [5], [0], 256) #image, distances, angles,
    dissimilarity = greycoprops(glcm,"dissimilarity")[0,0]
    correlation = greycoprops(glcm,"correlation")[0,0]
    contrast = greycoprops(glcm,"contrast")[0,0]
    homogeneity = greycoprops(glcm,"homogeneity")[0,0]
    ASM = greycoprops(glcm, "ASM")[0,0]
    energy = greycoprops(glcm,"energy")[0,0]
    GLCM=np.array((dissimilarity,correlation,contrast,homogeneity,ASM,energy))
    feat=GLCM
  elif(feat_type == "Histogram"):
    # Histogram- 10
    histogram=np.histogram(Channel)[0]
    feat=histogram
  elif(feat_type == "Fintesity"):
    # Intensity distribution features- 3
    fmean = np.mean(Channel)
    fskew = skew(Channel.reshape(-1))
    fkurtosis = kurtosis(Channel.reshape(-1))
    fintesity=np.array((fmean,fskew,fkurtosis))
    feat=fintesity
  elif(feat_type == "Fstat"):
    # Statistical based on probabilities - 3
    smoothness = 1- 1/(1 + np.var(Channel/2**8))
    histogram_prob,_ = np.histogram(Channel, bins=2**8,range=(0,256), density=True)
    uniformity = sum(np.power(histogram_prob,2))
    fentropy=entropy(histogram_prob)
    fstat=np.array((smoothness,uniformity,fentropy))
    feat=fstat
  elif(feat_type == "Gabour"):
    if (dermo_challenge) and (number_Windowing_type == 2):
      gabor_feats = compute_gabor_feats_mask(Channel, Gabour_kernels, mask_img)
    else:
      gabor_feats = compute_gabor_feats(Channel, Gabour_kernels)
    feat = gabor_feats
  else:
    print('Feature name is not correct!')
    feat = [];
  return feat

def global_all_channel_feat(I,feat_type):
  for c in range(I.shape[2]):
    if (c==0):
      features=one_channel_feat(I[:,:,c],feat_type)
    else:
      features=np.concatenate((features,one_channel_feat(I[:,:,c],feat_type)))
  return features



def window_all_channel_feat(I, feat_type, windowing = False, scale= None, mask_img = None):
  if(dermo_challenge): #For dermo challenge
    for c in range(I.shape[2]):
      if (c==0):
        features=one_channel_feat(I[:,:,c],feat_type, mask_img)
      else:
        features=np.concatenate((features,one_channel_feat(I[:,:,c],feat_type, mask_img)))
  else: # For histopathology challenge
    if(windowing): # Use rectangular windows
      h, w = I.shape[:2]
      for h_idx in range(scale): #Scale is Windows division
        for w_idx in range(scale):
          crop_img = I[int(h/scale)*h_idx:int(h/scale)*(h_idx+1), int(w/scale)*w_idx:int(w/scale)*(w_idx+1)]
          for c in range(crop_img.shape[2]):
            if (c==0):
              crop_features=one_channel_feat(crop_img[:,:,c],feat_type)
            else:
              crop_features=np.concatenate((crop_features,one_channel_feat(crop_img[:,:,c],feat_type)))
          if (h_idx==0 and w_idx==0):
            features =crop_features 
          else:
            features=np.concatenate((features,crop_features))
    else: # If there is not windowing for histopathology
      for c in range(I.shape[2]):
        if (c==0):
          features=one_channel_feat(I[:,:,c],feat_type)
        else:
          features=np.concatenate((features,one_channel_feat(I[:,:,c],feat_type)))                      
  return features



# Extract and Save the Data

In [0]:
def main_extraction(path,feat_type):
    img_dir=glob.glob(path)
    n_img=len(img_dir)
    print("Total number of images: ", n_img)
    n_img=n_img
    print("I am going to process: ",n_img)
    Flab = [];
    for p in range(n_img):
        if p%500==0:
          print('Iteration number:',p)
        img=cv2.imread(img_dir[p])
        if(dermo_challenge):
          # PREPROCESSING
          if (img[0,0,0]<30):
            print("Preprocessing in", img_dir[p])
            img=remove_black_focus(img)
        if(number_Windowing_type == 1):
          windowing_enabled = False
          window_scale = None
          mask_img = None

             
        if(number_Windowing_type == 2):
          windowing_enabled = True
          window_scale = 2
          mask_img = None
          if(dermo_challenge):
            #mask_img,erosion_img,dilation_img = create_segmented_ROI(img)
            mask_img = create_segmented_ROI(img)


        # COLOR SPACE TRANSFORMATION
          # Color space 3D- 5
        Ilab=cv2.cvtColor(img, cv2.COLOR_BGR2Lab)
        Ihsv=cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
        Irgb=img[:, :, : :-1]
        Ijch=rgb2jch(Irgb.reshape(-1, 3)) #CIECAM02  with 3 chanels is enought https://pypi.org/project/ciecam02/
        Ijch=Ijch.reshape(img.shape)
        Ijch=Ijch.astype('uint8')
        Ihybrid=hybrid_color_space(img)  
          # Color space 6D- 1    
        Isix=six_melanoma_colors(img)
          # Color space 4D- 1
        Itissue=tissue_segmentation(Ilab)

        # FEATURE CALCULATION
        # For the first image we allocate memory for the arrays
        if p==0:
          feat=window_all_channel_feat(Ilab, feat_type, windowing_enabled, window_scale, mask_img)
          Flab=np.empty((n_img,feat.shape[0]))
          Flab[0,:]=feat

          feat=window_all_channel_feat(Ihsv, feat_type, windowing_enabled, window_scale, mask_img)
          Fhsv=np.empty((n_img,feat.shape[0]))
          Fhsv[0,:]=feat

          feat=window_all_channel_feat(Irgb, feat_type, windowing_enabled, window_scale, mask_img)
          Frgb=np.empty((n_img,feat.shape[0]))
          Frgb[0,:]=feat

          feat=window_all_channel_feat(Ijch, feat_type, windowing_enabled, window_scale, mask_img)
          Fjch=np.empty((n_img,feat.shape[0]))
          Fjch[0,:]=feat

          if (dermo_challenge):
            Ftissue = []

            feat=window_all_channel_feat(Ihybrid, feat_type, windowing_enabled, window_scale, mask_img)
            Fhybrid=np.empty((n_img,feat.shape[0]))
            Fhybrid[0,:]=feat

            feat=window_all_channel_feat(Isix, feat_type, windowing_enabled, window_scale, mask_img)
            Fsix=np.empty((n_img,feat.shape[0]))
            Fsix[0,:]=feat           
          else:
            Fhybrid = []
            Fsix = []
            feat=window_all_channel_feat(Itissue, feat_type, windowing_enabled, window_scale, mask_img)
            Ftissue=np.empty((n_img,feat.shape[0]))
            Ftissue[0,:]=feat
        # For the rest of the loops         
        else:
          Flab[p,:]=window_all_channel_feat(Ilab, feat_type, windowing_enabled, window_scale, mask_img)
          Fhsv[p,:]=window_all_channel_feat(Ihsv, feat_type, windowing_enabled, window_scale, mask_img)
          Frgb[p,:]=window_all_channel_feat(Irgb, feat_type, windowing_enabled, window_scale, mask_img)  
          Fjch[p,:]=window_all_channel_feat(Ijch, feat_type, windowing_enabled, window_scale, mask_img)
          if (dermo_challenge):
            Fhybrid[p,:]=window_all_channel_feat(Ihybrid, feat_type, windowing_enabled, window_scale, mask_img)
            Fsix[p,:]=window_all_channel_feat(Isix, feat_type, windowing_enabled, window_scale, mask_img)
          else:       
            Ftissue[p,:]=window_all_channel_feat(Itissue, feat_type, windowing_enabled, window_scale, mask_img)
    return {"Flab":Flab, "Fhsv":Fhsv, "Frgb":Frgb, "Fjch":Fjch, "Ftissue":Ftissue, "Fhybrid":Fhybrid, "Fsix":Fsix}


def extract_and_save_test(feat_type,normalization,windowing):
  # Extract
  path4extract=path_data
  X_pos =main_extraction(path4extract,feat_type)
  print('Extraction done for: '+feat_type)
  print(X_pos["Flab"].shape)
  # Save
  if dermo_challenge: # There is not normalization step for dermo
    normalization=''
  path4save_general=path_general_feat+normalization+'/'+windowing+'/test/'
  path4save= path4save_general

  np.save(path4save+ 'Flab_'+feat_type+'.npy',X_pos["Flab"])
  np.save(path4save+ 'Fhsv_'+feat_type+'.npy',X_pos["Fhsv"])
  np.save(path4save+ 'Frgb_'+feat_type+'.npy',X_pos["Frgb"])
  np.save(path4save+ 'Fjch_'+feat_type+'.npy',X_pos["Fjch"])
  if (dermo_challenge):
    np.save(path4save+ 'Fhybrid_'+feat_type+'.npy',X_pos["Fhybrid"])
    np.save(path4save+ 'Fsix_'+feat_type+'.npy',X_pos["Fsix"])
  else:
    np.save(path4save+ 'Ftissue_'+feat_type+'.npy',X_pos["Ftissue"])


def extract_and_save_train_val(feat_type,normalization,windowing,bool_positives_cases):
  # Extract
  path4extract=path_data_pos if bool_positives_cases  else path_data_neg
  X_pos =main_extraction(path4extract,feat_type)
  print('Extraction done for: '+feat_type)
  print(X_pos["Flab"].shape)
  # Save
  if dermo_challenge: # There is not normalization step for dermo
    normalization=''
  if(number_Dataset_type = 1):
    path4save_general=path_general_feat+normalization+'/'+windowing+'/train/'
  if(number_Dataset_type = 2):
    path4save_general=path_general_feat+normalization+'/'+windowing+'/val/'
  path4save= path4save_general+'pos/' if bool_positives_cases  else path4save_general+'neg/'

  np.save(path4save+ 'Flab_'+feat_type+'.npy',X_pos["Flab"])
  np.save(path4save+ 'Fhsv_'+feat_type+'.npy',X_pos["Fhsv"])
  np.save(path4save+ 'Frgb_'+feat_type+'.npy',X_pos["Frgb"])
  np.save(path4save+ 'Fjch_'+feat_type+'.npy',X_pos["Fjch"])
  if (dermo_challenge):
    np.save(path4save+ 'Fhybrid_'+feat_type+'.npy',X_pos["Fhybrid"])
    np.save(path4save+ 'Fsix_'+feat_type+'.npy',X_pos["Fsix"])
  else:
    np.save(path4save+ 'Ftissue_'+feat_type+'.npy',X_pos["Ftissue"])

  

In [0]:
from google.colab import drive
drive.mount("/content/drive/", force_remount=True)
print(windowing_type)
if isinstance(number_Feat_type,int):
  feat_type=switcher_feat.get(number_Feat_type, "Invalid feature type")
  if(number_Dataset_type == 1) OR (number_Dataset_type == 2):
    extract_and_save_train_val(feat_type,normalization_type,windowing_type,bool_positives_cases)
  if(number_Dataset_type == 3):
    extract_and_save_test(feat_type,normalization_type,windowing_type)
else:
  for i in range(len(number_Feat_type)):
    feat_type=switcher_feat.get(number_Feat_type[i], "Invalid feature type")
    if(number_Dataset_type == 1) OR (number_Dataset_type == 2):
      extract_and_save_train_val(feat_type,normalization_type,windowing_type,bool_positives_cases)
    if(number_Dataset_type == 3):
      extract_and_save_test(feat_type,normalization_type,windowing_type)

Mounted at /content/drive/
Two
Total number of images:  1015
I am going to process:  1015
Iteration number: 0
Iteration number: 1
Iteration number: 2
Preprocessing in /content/drive/My Drive/Colab Notebooks/CAD Project/Dermo Challenge/test/xx0519.jpg
Iteration number: 3
Iteration number: 4
Iteration number: 5
Iteration number: 6
Iteration number: 7
Iteration number: 8
Iteration number: 9
Iteration number: 10
Iteration number: 11
Iteration number: 12
Iteration number: 13
Iteration number: 14
Preprocessing in /content/drive/My Drive/Colab Notebooks/CAD Project/Dermo Challenge/test/xx0526.jpg
Iteration number: 15
Iteration number: 16
Iteration number: 17
Iteration number: 18
Iteration number: 19
Iteration number: 20
Iteration number: 21
Iteration number: 22
Iteration number: 23
Iteration number: 24
Iteration number: 25
Iteration number: 26
Iteration number: 27
Iteration number: 28
Iteration number: 29
Iteration number: 30
Iteration number: 31
Iteration number: 32
Preprocessing in /content